# 🔥 Enhanced Viral Shorts Generator

**Features:**
- ✨ TikTok-style word-by-word captions with professional fonts
- 🌈 Animated rainbow gradient background (no black bars!)
- 🎵 Background music support
- 🤖 AI-powered hook detection to find the best 60s clip

---

## 📦 Step 1: Install Dependencies

In [ ]:
!pip install -q yt-dlp openai-whisper torch numpy pysrt
!apt-get update -qq
!apt-get install -qq ffmpeg fonts-dejavu

## ⚙️ Step 2: Configuration

Edit these settings for your video:

In [ ]:
# =============================================================================
# 🎬 CONFIGURATION - EDIT THESE!
# =============================================================================

# Your YouTube URL (or set to None if using local file)
YOUTUBE_URL = "https://www.youtube.com/watch?v=BWGu7piZMcQ"  # <-- CHANGE THIS!

# Or use a local video file (upload to Kaggle first)
LOCAL_VIDEO = None  # e.g., "/kaggle/input/your-dataset/video.mp4"

# Background music (upload your own or set to None)
MUSIC_FILE = None # e.g., "/kaggle/input/your-dataset/music.mp3"
DOWNLOAD_SAMPLE_MUSIC = True  # Download free music if no file provided

# Video settings
CLIP_DURATION = 38  # Duration in seconds (60 for TikTok, 90 for Reels)
OUTPUT_WIDTH = 1080
OUTPUT_HEIGHT = 1920

# Audio mix
MUSIC_VOLUME = 0.15  # Background music volume (0.0 - 1.0)
ORIGINAL_VOLUME = 1.0  # Original audio volume

# Whisper model ('tiny', 'base', 'small', 'medium', 'large')
# Larger = more accurate but slower
WHISPER_MODEL = "medium"

print("✅ Configuration loaded!")

## 🎨 Step 3: Choose Subtitle Style

Run this cell and pick your favorite style!

In [ ]:
# =============================================================================
# 🎨 SUBTITLE STYLE PRESETS
# =============================================================================

STYLES = {
    # Classic TikTok - Bold white with black outline
    "classic": {
        "font": "Montserrat-Bold",
        "font_size": 12,
        "primary_color": "&H00FFFFFF",
        "outline_color": "&H00000000",
        "back_color": "&H00000000",
        "outline": 1,
        "shadow": 1,
        "bold": 1,
        "alignment": 2,
        "margin_v": 100,
    },
    
    # Neon Glow - Cyan with magenta outline
    "neon": {
        "font": "Impact",
        "font_size": 22,
        "primary_color": "&H00FFFF00",
        "outline_color": "&H00FF00FF",
        "back_color": "&H00000000",
        "outline": 5,
        "shadow": 3,
        "bold": 1,
        "alignment": 2,
        "margin_v": 100,
    },
    
    # Bold Yellow - High contrast
    "yellow": {
        "font": "Montserrat-Bold",
        "font_size": 22,
        "primary_color": "&H0000FFFF",
        "outline_color": "&H00000000",
        "back_color": "&H00000000",
        "outline": 5,
        "shadow": 3,
        "bold": 1,
        "alignment": 2,
        "margin_v": 100,
    },
    
    # Fire Style - Orange
    "fire": {
        "font": "Impact",
        "font_size": 22,
        "primary_color": "&H000080FF",
        "outline_color": "&H00000080",
        "back_color": "&H00000000",
        "outline": 4,
        "shadow": 2,
        "bold": 1,
        "alignment": 2,
        "margin_v": 110,
    },
    
    # MrBeast Style - Bold green
    "mrbeast": {
        "font": "Impact",
        "font_size": 22,
        "primary_color": "&H0000FF00",
        "outline_color": "&H00000000",
        "back_color": "&H00000000",
        "outline": 6,
        "shadow": 4,
        "bold": 1,
        "alignment": 2,
        "margin_v": 80,
    },
    
    # Minimal - Clean white
    "minimal": {
        "font": "Helvetica-Bold",
        "font_size": 22,
        "primary_color": "&H00FFFFFF",
        "outline_color": "&H80000000",
        "back_color": "&H00000000",
        "outline": 2,
        "shadow": 0,
        "bold": 1,
        "alignment": 2,
        "margin_v": 150,
    },
    
    # Boxed - Text with background
    "boxed": {
        "font": "Roboto-Bold",
        "font_size": 22,
        "primary_color": "&H00FFFFFF",
        "outline_color": "&H00000000",
        "back_color": "&HCC000000",
        "outline": 0,
        "shadow": 0,
        "bold": 1,
        "alignment": 2,
        "margin_v": 130,
    },
    
    # Aesthetic Pink
    "aesthetic": {
        "font": "Georgia-Bold",
        "font_size": 22,
        "primary_color": "&H00CBC0FF",
        "outline_color": "&H00800080",
        "back_color": "&H00000000",
        "outline": 3,
        "shadow": 2,
        "bold": 1,
        "alignment": 2,
        "margin_v": 140,
    },
}

# =============================================================================
# 👇 CHOOSE YOUR STYLE HERE!
# =============================================================================
SELECTED_STYLE = "boxed"  # Options: classic, neon, yellow, fire, mrbeast, minimal, boxed, aesthetic

SUBTITLE_STYLE = STYLES[SELECTED_STYLE]

print(f"🎨 Selected style: {SELECTED_STYLE.upper()}")
print(f"   Font: {SUBTITLE_STYLE['font']}, Size: {SUBTITLE_STYLE['font_size']}")

## 📚 Step 4: Import Libraries & Define Functions

In [ ]:
import whisper
import subprocess
import pysrt
import re
import os
from IPython.display import Video, display, HTML

print("✅ Libraries imported!")

In [ ]:
# =============================================================================
# 📥 DOWNLOAD VIDEO
# =============================================================================
def download_video(url, output):
    """Download video from YouTube"""
    print("📥 Downloading video...")
    subprocess.run([
        "yt-dlp",
        "-f", "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best",
        "--merge-output-format", "mp4",
        "-o", output,
        url
    ], check=True)
    print("✅ Download complete!")

# =============================================================================
# 🎤 TRANSCRIBE VIDEO
# =============================================================================
def transcribe_video(video_path, model_size="base"):
    """Transcribe video using Whisper"""
    print(f"🎤 Transcribing video with '{model_size}' model...")
    model = whisper.load_model(model_size)
    result = model.transcribe(video_path, word_timestamps=True)
    print("✅ Transcription complete!")
    return result["segments"]

# =============================================================================
# 🧠 AI HOOK DETECTION
# =============================================================================
def score_segment(text):
    """Score text for viral potential"""
    score = 0
    text_lower = text.lower()
    
    if "?" in text:
        score += 3
    if re.search(r"\b(you|your|you're|you've)\b", text_lower):
        score += 2
    
    power_words = [
        "secret", "truth", "nobody", "mistake", "never", "always",
        "shocking", "amazing", "incredible", "unbelievable", "crazy",
        "hack", "trick", "tip", "revealed", "hidden", "finally",
        "stop", "don't", "must", "need", "now", "today", "free"
    ]
    for word in power_words:
        if word in text_lower:
            score += 2
    
    if re.search(r"\b\d+\b", text):
        score += 1
    
    word_count = len(text.split())
    if 10 <= word_count <= 30:
        score += 1
    
    return score

def find_best_segment(segments, duration=60):
    """Find the most engaging segment"""
    print("🔍 Finding most engaging segment...")
    
    window = []
    scores = []
    
    for seg in segments:
        window.append(seg)
        while window and window[-1]["end"] - window[0]["start"] > duration:
            window.pop(0)
        if window:
            combined_text = " ".join(s["text"] for s in window)
            scores.append((score_segment(combined_text), window[0]["start"]))
    
    if not scores:
        return 0
    
    best_start = max(scores, key=lambda x: x[0])[1]
    print(f"✅ Best segment starts at {best_start:.2f}s")
    return best_start

print("✅ Functions defined!")

In [ ]:
# =============================================================================
# 🌈 CREATE RAINBOW BACKGROUND
# =============================================================================
def create_rainbow_background(duration, width, height, output):
    """Create animated blurred rainbow gradient background"""
    print("🌈 Creating animated rainbow background...")
    
    filter_cmd = (
        f"color=s={width}x{height}:c=red:d={duration},"
        f"hue=H=t*60:s=2,"
        f"boxblur=luma_radius=100:luma_power=3,"
        f"eq=brightness=0.05:saturation=1.2"
    )
    
    subprocess.run([
        "ffmpeg", "-y",
        "-f", "lavfi",
        "-i", filter_cmd,
        "-t", str(duration),
        "-c:v", "libx264",
        "-preset", "fast",
        "-crf", "23",
        "-pix_fmt", "yuv420p",
        output
    ], check=True)
    
    print("✅ Rainbow background created!")

# =============================================================================
# ✂️ CUT CLIP
# =============================================================================
def cut_clip(input_video, start_time, duration, output):
    """Cut the best segment from the video"""
    print("✂️ Cutting clip...")
    subprocess.run([
        "ffmpeg", "-y",
        "-ss", str(start_time),
        "-i", input_video,
        "-t", str(duration),
        "-c:v", "libx264",
        "-c:a", "aac",
        "-preset", "fast",
        output
    ], check=True)
    print("✅ Clip cut!")

# =============================================================================
# 🎨 COMBINE VIDEO WITH RAINBOW
# =============================================================================
def combine_with_rainbow(clip, rainbow_bg, output, width, height):
    """Overlay video on rainbow background"""
    print("🎨 Combining video with rainbow background...")
    
    filter_complex = (
        f"[1:v]scale=-1:{height}:force_original_aspect_ratio=decrease[scaled];"
        f"[0:v][scaled]overlay=(W-w)/2:(H-h)/2:shortest=1[outv]"
    )
    
    subprocess.run([
        "ffmpeg", "-y",
        "-i", rainbow_bg,
        "-i", clip,
        "-filter_complex", filter_complex,
        "-map", "[outv]",
        "-map", "1:a?",
        "-c:v", "libx264",
        "-c:a", "aac",
        "-preset", "fast",
        "-crf", "23",
        "-shortest",
        output
    ], check=True)
    
    print("✅ Combined!")

print("✅ Video processing functions defined!")

In [ ]:
# =============================================================================
# 📝 CREATE SUBTITLES
# =============================================================================
def create_subtitles(segments, start_time, duration, output_srt):
    """Create word-by-word TikTok-style subtitles"""
    print("📝 Creating subtitles...")
    
    subs = pysrt.SubRipFile()
    index = 1
    end_time = start_time + duration
    
    for seg in segments:
        seg_start = seg["start"]
        seg_end = seg["end"]
        
        if seg_start < start_time or seg_start > end_time:
            continue
        
        if "words" in seg:
            for word_info in seg["words"]:
                word = word_info.get("word", "").strip()
                if not word:
                    continue
                
                w_start = word_info["start"] - start_time
                w_end = word_info["end"] - start_time
                
                if w_start < 0 or w_start > duration:
                    continue
                
                s = pysrt.SubRipItem()
                s.index = index
                s.start.seconds = max(0, w_start)
                s.end.seconds = min(duration, w_end)
                s.text = word.upper()
                subs.append(s)
                index += 1
        else:
            words = seg["text"].strip().split()
            if not words:
                continue
            
            seg_duration = seg_end - seg_start
            word_duration = seg_duration / len(words)
            
            for i, word in enumerate(words):
                word_start = (seg_start - start_time) + (i * word_duration)
                word_end = word_start + word_duration
                
                if word_start < 0 or word_start > duration:
                    continue
                
                s = pysrt.SubRipItem()
                s.index = index
                s.start.seconds = max(0, word_start)
                s.end.seconds = min(duration, word_end)
                s.text = word.upper()
                subs.append(s)
                index += 1
    
    subs.save(output_srt)
    print(f"✅ Created {len(subs)} subtitle entries!")
    return output_srt

# =============================================================================
# 🔥 BURN SUBTITLES
# =============================================================================
def burn_subtitles(input_video, srt_file, output, style):
    """Burn TikTok-style subtitles onto video"""
    print("🔥 Burning subtitles...")
    
    force_style = (
        f"FontName={style['font']},"
        f"FontSize={style['font_size']},"
        f"PrimaryColour={style['primary_color']},"
        f"OutlineColour={style['outline_color']},"
        f"BackColour={style['back_color']},"
        f"BorderStyle=1,"
        f"Outline={style['outline']},"
        f"Shadow={style['shadow']},"
        f"Bold={style['bold']},"
        f"Alignment={style['alignment']},"
        f"MarginV={style['margin_v']}"
    )
    
    srt_escaped = srt_file.replace(":", r"\:").replace("'", r"\'")
    subtitle_filter = f"subtitles={srt_escaped}:force_style='{force_style}'"
    
    subprocess.run([
        "ffmpeg", "-y",
        "-i", input_video,
        "-vf", subtitle_filter,
        "-c:v", "libx264",
        "-c:a", "copy",
        "-preset", "fast",
        "-crf", "23",
        output
    ], check=True)
    
    print("✅ Subtitles burned!")

print("✅ Subtitle functions defined!")

In [ ]:
# =============================================================================
# 🎵 MUSIC FUNCTIONS
# =============================================================================
def download_sample_music(output_file="music.mp3"):
    """Download royalty-free background music"""
    print("🎵 Downloading sample music...")
    
    # Royalty-free music from Pixabay
    urls = [
        "https://cdn.pixabay.com/download/audio/2022/10/25/audio_946b0939c8.mp3",
        "https://cdn.pixabay.com/download/audio/2022/03/15/audio_c8c8a73467.mp3",
    ]
    
    for url in urls:
        try:
            subprocess.run(["wget", "-q", "-O", output_file, url], check=True, timeout=30)
            print(f"✅ Downloaded music to {output_file}")
            return True
        except:
            continue
    
    print("⚠️ Could not download music")
    return False

def add_music(input_video, music_file, output, music_vol=0.15, original_vol=1.0):
    """Mix background music with original audio"""
    print("🎵 Adding background music...")
    
    if not os.path.exists(music_file):
        print(f"⚠️ Music file not found, skipping...")
        subprocess.run(["cp", input_video, output])
        return
    
    # Get duration
    result = subprocess.run([
        "ffprobe", "-v", "error",
        "-show_entries", "format=duration",
        "-of", "default=noprint_wrappers=1:nokey=1",
        input_video
    ], capture_output=True, text=True)
    duration = float(result.stdout.strip())
    
    filter_complex = (
        f"[1:a]aloop=loop=-1:size=2e+09,atrim=0:{duration},"
        f"volume={music_vol}[music];"
        f"[0:a]volume={original_vol}[original];"
        f"[original][music]amix=inputs=2:duration=first:dropout_transition=2[aout]"
    )
    
    subprocess.run([
        "ffmpeg", "-y",
        "-i", input_video,
        "-i", music_file,
        "-filter_complex", filter_complex,
        "-map", "0:v",
        "-map", "[aout]",
        "-c:v", "copy",
        "-c:a", "aac",
        "-b:a", "192k",
        "-shortest",
        output
    ], check=True)
    
    print("✅ Music added!")

print("✅ Music functions defined!")

## 🚀 Step 5: Run the Generator!

This is the main cell - run it to create your viral short!

In [ ]:
# =============================================================================
# 🚀 MAIN PIPELINE
# =============================================================================

# File paths
VIDEO = "video.mp4"
CLIP = "clip.mp4"
RAINBOW_BG = "rainbow_bg.mp4"
COMBINED = "combined.mp4"
SUBTITLED = "subtitled.mp4"
FINAL = "final_short.mp4"
MUSIC = "music.mp3"

print("="*60)
print("🎬 VIRAL SHORTS GENERATOR")
print("="*60)

# Step 1: Get video
if LOCAL_VIDEO and os.path.exists(LOCAL_VIDEO):
    VIDEO = LOCAL_VIDEO
    print(f"📁 Using local video: {VIDEO}")
elif YOUTUBE_URL and "VIDEO_ID" not in YOUTUBE_URL:
    download_video(YOUTUBE_URL, VIDEO)
else:
    raise ValueError("❌ Please set YOUTUBE_URL or LOCAL_VIDEO in the configuration!")

# Step 2: Transcribe
segments = transcribe_video(VIDEO, WHISPER_MODEL)

# Step 3: Find best segment
best_start = find_best_segment(segments, CLIP_DURATION)

# Step 4: Create rainbow background
create_rainbow_background(CLIP_DURATION + 1, OUTPUT_WIDTH, OUTPUT_HEIGHT, RAINBOW_BG)

# Step 5: Cut clip
cut_clip(VIDEO, best_start, CLIP_DURATION, CLIP)

# Step 6: Combine with rainbow
combine_with_rainbow(CLIP, RAINBOW_BG, COMBINED, OUTPUT_WIDTH, OUTPUT_HEIGHT)

# Step 7: Create and burn subtitles
create_subtitles(segments, best_start, CLIP_DURATION, "subs.srt")
burn_subtitles(COMBINED, "subs.srt", SUBTITLED, SUBTITLE_STYLE)

# Step 8: Add music
if MUSIC_FILE and os.path.exists(MUSIC_FILE):
    add_music(SUBTITLED, MUSIC_FILE, FINAL, MUSIC_VOLUME, ORIGINAL_VOLUME)
elif DOWNLOAD_SAMPLE_MUSIC:
    if download_sample_music(MUSIC):
        add_music(SUBTITLED, MUSIC, FINAL, MUSIC_VOLUME, ORIGINAL_VOLUME)
    else:
        subprocess.run(["cp", SUBTITLED, FINAL])
else:
    subprocess.run(["cp", SUBTITLED, FINAL])

# Cleanup
for f in [CLIP, RAINBOW_BG, COMBINED, SUBTITLED]:
    if os.path.exists(f):
        os.remove(f)

print("\n" + "="*60)
print("🔥🔥🔥 VIRAL SHORT CREATED! 🔥🔥🔥")
print("="*60)
print(f"📱 Output: {FINAL}")
print(f"📐 Resolution: {OUTPUT_WIDTH}x{OUTPUT_HEIGHT}")
print(f"⏱️ Duration: {CLIP_DURATION}s")

## 📺 Step 6: Preview Your Video

In [ ]:
# Preview the final video (may not work on all platforms)
if os.path.exists(FINAL):
    display(Video(FINAL, width=360))
else:
    print("❌ Video not found!")